## Preparación de datos

En primer lugar importamos pandas para poder navegar por el dataset y hacer las transformaciones que consideremos necesarias

In [ ]:
import pandas as pd

#Importamos los datasets  y los leemos con pandas desde las rutas en las que están los datos.
estimar_ = "Estimar_uh2020.txt"
modelar_ = "Modelar_uh2020.txt"
estimar = pd.read_csv(estimar_, sep='|')
modelar = pd.read_csv(modelar_, sep='|')
datos_train=modelar

La mayor dificultad que presenta el conjunto de datos es el desbalanceo de las clases. Como podemos observar en el resultado del siguiente código, hay una gran desproporción entre la cantidad de filas de unas clases y otras. El modelo debe ser generalista y mostrar buenos resultados para cualquier conjunto de datos, así que asignaremos un mayor peso a las clases con menor número de ocurrencias cuando lo instanciemos.

In [ ]:
modelar.groupby("CLASE").count()["ID"]

Cuando evaluemos el modelo, queremos probarlo con un dataset balanceado, ya que si lo hacemos con uno desbalanceado los resultados pueden verse influidos por la frecuencia de aparición de la clase mayoritaria. Por esto, creamos el dataframe "balanceado" y lo almacenamos en un fichero, para disponer de él siempre que queramos. Este fichero contiene 150 filas de cada clase y será utilizado como conjunto de datos de test.

In [ ]:
clases={"AGRICULTURE":338,
        "INDUSTRIAL": 4490,
        "OFFICE": 1828,
        "OTHER": 1332,
        "PUBLIC": 2976,
        "RESIDENTIAL": 90173,
        "RETAIL": 2093}

aux=modelar
for index, line in aux.iterrows():
    if clases[line["CLASE"]]>150:
        aux = aux.drop(index)
        print(modelar.shape)
        clases[line["CLASE"]] = clases[line["CLASE"]]-1

aux.to_csv("balanceado.csv")

In [ ]:
balanceado = pd.read_csv("balanceado.csv",index_col=0)

Por otra parte vamos a ver en primer lugar valores estadísticos básicos de cada variable para hacernos una idea general y posteriormente veremos si hay valores nulos y donde están estos.

In [ ]:
#Vamos a ver valores estadísticos básicos de las variables para hacernos una idea en general de los datos.
modelar.describe()

In [ ]:
#Queremos saber si hay valores nulos, ya que esto afectará a nuestra predicción.
modelar.isnull().any()

In [ ]:
#hacemos lo mismo con los valores a estimar
estimar.describe()

In [ ]:
estimar.isnull().any()

In [ ]:
#Queremos ver si los valores nulos son únicos de una clase o están repartidos por todas las clases
modelar[modelar["CADASTRALQUALITYID"].isnull()]

In [ ]:
#Podemos ver que las filas afectadas con valores nulos son lasmismas para ambas variables 
estimar[estimar["MAXBUILDINGFLOOR"].isnull()]

In [ ]:
#La mayoría están en agriculture, con este código vamos a ver la distribución de los valores nulos en las clases
modelar[modelar["MAXBUILDINGFLOOR"].isnull()].groupby('CLASE')["CLASE"].count()

En su mayoría están en la clase Agriculture,que justamente es la clase de la que menos valores tenemos, por lo que en una primera instancia vamos a intentar no eliminarlos a no ser que no sea posible. 
Para ello veremos si este valor depende de los demás parámetros para predecirlo y en caso contrario, lo eliminaremos o haremos una reducción de variables para no tenerlo en cuenta.

In [ ]:
modelar[modelar["CLASE"]=="AGRICULTURE"].groupby('CADASTRALQUALITYID')['CADASTRALQUALITYID'].count()

In [ ]:

#Creamos dataframe con los valores que queremos predecir, los nulos
agr_predict=modelar[modelar["CADASTRALQUALITYID"].isnull()]
#Dataframe con datos para hacer la predicción, los no nulos
agr_data=modelar[~modelar["CADASTRALQUALITYID"].isnull()]
#separamos datos del target
X=agr_data.drop("CADASTRALQUALITYID",axis=1)._get_numeric_data()
y=agr_data["CADASTRALQUALITYID"]


In [ ]:
#Primero probamos con un árbol de decisión sencillo para ver si obtenemos un resultado decente
from sklearn.model_selection import GridSearchCV, cross_validate
from sklearn.tree import DecisionTreeClassifier

t = DecisionTreeClassifier(max_depth=5)
cross_validate(t, X, y,return_train_score=True)


In [ ]:
#Ahora con una regresión
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

logr = Pipeline([('std', StandardScaler()), ('lr', LogisticRegression(max_iter=1000,solver="saga"))])
cross_validate(logr, X, y,return_train_score=True)


No obtenemos valores aceptables, por lo que vamos a seguir la siguiente estrategia:
- Primero entrenaremos mediante grid search varios modelos hasta quedarnos con el mejor, dando el valor 0 a las filas con valor nulo.
- Haremos una reducción de variables para no caer en la maldición de la dimensionalidad, de tal modo que eligiremos según un test chi2 las 40 mejores
- Seguiremos reduciendo el número de variables hasta que la calidad del modelo se vea comprometida, de este modo, buscamos que las variables con valor nulo no estén entre las más significativas y así no comprometer el dataset eliminando filas o dando valores aleatorios a estas. 



Otro paso importante en el procesamiento previo de los datos es la categorización de las variables no numéricas. En este caso, solamente hay una, "CADASTRALQUALITYID", que puede tomar valores numéricos del 0 al 9 junto con los carácteres "A", "B" y "C". 

In [ ]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
#Los relleno como 0 para poder realizar el test, sin embargo esto no tiene problema ya que posteriormente no utilizaremos esta variable para la predicción.
#De este modo podemos contar con etsa fila de datos sin necesidad de eliminarla
modelar=modelar.fillna(0)#rellenamos nan con 0
modelar["CADASTRALQUALITYID"] = modelar["CADASTRALQUALITYID"].astype(str)
encoder.fit(modelar["CADASTRALQUALITYID"])#convertimos a valor numérico los valores String
modelar["CADASTRALQUALITYID"] = encoder.transform(modelar["CADASTRALQUALITYID"])

#Obtengo el dataset balanceado para hacer el test:
balanceado = balanceado.fillna(0)
balanceado["CADASTRALQUALITYID"] = balanceado["CADASTRALQUALITYID"].astype(str)
encoder.fit(balanceado["CADASTRALQUALITYID"])
balanceado["CADASTRALQUALITYID"] = encoder.transform(balanceado["CADASTRALQUALITYID"])

Para generar el conjunto de datos de entrenamiento, eliminamos del dataframe "datos" las filas contenidas en "balanceado", pues no deberíamos testear el modelo con datos utilizados en su entrenamiento. Para asegurarnos que ambos conjuntos son disjuntos, podemos mostrar su forma por pantalla. Podemos observar que el número de filas de "datos_train" es igual a la diferencia del número de filas de "datos" y "balanceado"

In [ ]:
datos_train = modelar.loc[modelar.index.difference(balanceado.index), ]
display(modelar.shape,datos_train.shape,balanceado.shape)

In [ ]:
X = datos_train.drop("CLASE", axis=1).drop("ID",axis=1)
y= datos_train["CLASE"]
X_test = balanceado.drop("CLASE", axis=1).drop("ID",axis=1)
y_test = balanceado["CLASE"]

Vamos a utilizar un RandomForestClassifier con estos parámetros ya que tras haber buscado varios modelos y parámetros mediante GridSearch este ha sido el mejor. El GridSearch entrenado se puede encontrar en el adjunto GridSearch.joblib. Utilizando un scoring objetivo de "precision macro" los parámetros que mejores resultados arrojaron fueron n_estimators=700 y max_depth=50. Además, aprovechamos la posibilidad que ofrece de balancear los diferentes pesos con la opción de class_weight.

In [ ]:
#De todos los modelos que hemos probado y todos los parámetros probados mediante GridSearch nos hemos quedado con este modelo como el que mejor resultado nos ha dado.
#Este es el resultado con todas las variables
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV, cross_validate



rf = RandomForestClassifier(n_estimators=700,max_depth=50,min_samples_leaf=20,min_samples_split=20, criterion="entropy",class_weight='balanced')
rf.fit(X,y)
print(classification_report(y_test, rf.predict(X_test)))

In [ ]:
import matplotlib.pyplot as plt
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    # print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=90)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix
cnf_matrix = confusion_matrix(y_test, rf.predict(X_test))
# Plot normalized confusion matrix
fig = plt.figure()
fig.set_size_inches(10, 10, forward=True)
plot_confusion_matrix(cnf_matrix, classes=np.asarray(np.unique(y_test)), normalize=True,
                      title='Normalized confusion matrix')

Ahora procedeemos  a hacer una reducción de variables,eligiendo solo 30, de tal modo que si las variables con valor nulo no están entre las más importantes y la calidad no se ve comprometida nos quedaremos con este modelo, ya que será más sencillo y no nos preocuparemos por no tener dichas variables. Como se puede observar en el resultado, no están entre las 30 variables el CADASTRALQUALITY ni el MAXBUILDINGFLOOR, por lo que las variables que antes habíamos rellenado con 0 no afectarán al resultado final y por tanto a la calidad.

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

variables=modelar.loc[:, modelar.columns != 'CLASE']
targets=modelar["CLASE"]

chi2 = SelectKBest(chi2, k=30).fit(variables._get_numeric_data(), targets)
for (col, sel) in zip(variables._get_numeric_data().columns.values, chi2.get_support()):
    if sel:
        print(col)
seleccion = chi2.transform(variables._get_numeric_data())

In [ ]:
from sklearn.pipeline import Pipeline

rf = RandomForestClassifier(n_estimators=700,max_depth=50,class_weight='balanced')
estimator= Pipeline([("reduccion",chi2),("rf",rf)])
estimator.fit(X,y)
print(classification_report(y_test, estimator.predict(X_test)))

Una vez hemos seleccionado ya el modelo, lo aplicaremos a los datos a predecir para posteriormente montar el CSV a entregar.

In [ ]:
ID_pred = estimar['ID']
estimar = estimar.drop("ID",axis=1)

In [ ]:
estimar=estimar.fillna(0)#rellenamos nan con 0
estimar["CADASTRALQUALITYID"] = estimar["CADASTRALQUALITYID"].astype(str)
encoder.fit(estimar["CADASTRALQUALITYID"])
estimar["CADASTRALQUALITYID"] = encoder.transform(estimar["CADASTRALQUALITYID"])

In [ ]:
Clases_pred = estimator.predict(estimar)

In [ ]:
Clases_pred = pd.Series(Clases_pred)
entrega = pd. concat([ID_pred, Clases_pred], axis=1) 

In [ ]:
entrega.columns = ['ID','CLASE']

In [ ]:
entrega

In [ ]:
entrega.to_csv('_Los pentahos.txt',sep='|')